Построение модели Хестона для ценообразования опциона на индекс S&P500 по историческим данным

In [67]:
import QuantLib as ql
import pandas as pd
from datetime import datetime
import numpy as np

Чтение данных со страйками

In [3]:
strikes_df = pd.read_csv('strikes.csv', sep='\t', header=None)
strikes_df = strikes_df[0].str.split(';', expand=True)

In [4]:
new_header = strikes_df.iloc[0]
strikes_df = strikes_df[1:]
strikes_df.columns = new_header

In [5]:
strikes_df.rename(columns = {'':'date'}, inplace = True)

In [6]:
strikes_df

,date,SPX10P1W=R,SPX15P1W=R,SPX20P1W=R,SPX25P1W=R,SPX30P1W=R,SPX35P1W=R,SPX40P1W=R,SPX45P1W=R,SPX1WO=R,...,SPX45P24M=R,SPX24MO=R,SPX45C24M=R,SPX40C24M=R,SPX35C24M=R,SPX30C24M=R,SPX25C24M=R,SPX20C24M=R,SPX15C24M=R,SPX10C24M=R
1,15.08.2019,"2737,9","2761,38","2779,16","2793,82","2806,52","2817,93","2828,43","2838,3","2847,62",...,"2857,72","2881,39","2968,2","3050,68","3131,84","3213,04","3296,06","3383,74","3482,22","3611,57"
2,14.08.2019,"2725,94","2750,04","2768,3","2783,34","2796,39","2808,1","2818,88","2829,03","2838,47",...,"2853,51","2875,97","2962,79","3045,25","3126,38","3207,58","3290,66","3378,57","3478,01","3610,23"
3,13.08.2019,"2827,15","2854,99","2874,48","2889,46","2901,67","2912,04","2921,12","2929,32","2937,36",...,"2943,77","2970,09","3054,12","3133,9","3212,27","3290,77","3371,55","3458,28","3558,97","3694,05"
4,12.08.2019,"2739,01","2773,5","2797,92","2816,91","2832,54","2845,93","2857,75","2868,47","2877,95",...,"2891,04","2912,06","2998,11","3079,61","3159,66","3239,66","3321,5","3408,36","3507,77","3642,12"
5,09.08.2019,"2803,9","2833,92","2854,64","2870,37","2883,04","2893,68","2902,93","2911,17","2918,63",...,"2930,37","2953,01","3037,51","3117,63","3196,33","3275,08","3355,94","3442,48","3542,77","3678,28"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,11.09.2017,"2423,44","2441,61","2453,13","2461,43","2467,96","2473,43","2478,26","2482,68","2486,81",...,"2489,87","2509,57","2575,45","2637,92","2699,42","2761,34","2825,54","2894,83","2974,35","3075,82"
487,08.09.2017,"2386,97","2407,72","2421,66","2432,01","2440,2","2446,99","2452,85","2458,08","2462,89",...,"2463,5","2484,27","2550,3","2612,91","2674,46","2736,34","2800,29","2869,02","2947,47",3047
488,07.09.2017,"2402,66","2420,19","2431,93","2440,63","2447,49","2453,16","2458,04","2462,37","2466,36",...,"2466,16","2486,42","2551,71","2613,54","2674,25","2735,21","2798,21","2866,04","2943,79","3043,02"
489,06.09.2017,"2419,35","2432,45","2440,5","2446,41","2451,22","2455,38","2459,14","2462,66","2466,04",...,"2466,49","2485,95","2551,05","2612,63","2673,08","2733,8","2796,58","2864,22","2941,81","3040,88"


Чтение данных с iv

In [7]:
iv_df = pd.read_csv('iv.csv', sep='\t', header=None)
iv_df = iv_df[0].str.split(';', expand=True)

In [8]:
new_header = iv_df.iloc[0]
iv_df = iv_df[1:]
iv_df.columns = new_header

In [13]:
iv_df.rename(columns = {'':'date'}, inplace = True)

In [14]:
iv_df

,date,SPX10P1W=R,SPX15P1W=R,SPX20P1W=R,SPX25P1W=R,SPX30P1W=R,SPX35P1W=R,SPX40P1W=R,SPX45P1W=R,SPX1WO=R,...,SPX45P24M=R,SPX24MO=R,SPX45C24M=R,SPX40C24M=R,SPX35C24M=R,SPX30C24M=R,SPX25C24M=R,SPX20C24M=R,SPX15C24M=R,SPX10C24M=R
1,15.08.2019,"22,23","21,52","20,98","20,53","20,14","19,79","19,46","19,15","18,86",...,"17,45","17,06","16,68","16,1","15,54","14,97","14,4","13,82","13,23","12,75"
2,14.08.2019,"22,91","22,18","21,62","21,16","20,76","20,39","20,06","19,75","19,45",...,"17,44","17,05","16,68","16,1","15,54","14,97","14,4","13,83","13,26","12,81"
3,13.08.2019,"21,61","19,85","18,56","17,56","16,73","16,04","15,46","14,97","14,54",...,"16,53","16,13","15,74","15,18","14,63","14,1","13,58","13,07","12,61","12,27"
4,12.08.2019,"28,18","26,05","24,5","23,27","22,25","21,39","20,64","19,98","19,4",...,"17,05","16,68","16,32","15,74","15,18","14,62","14,06","13,5","12,97","12,58"
5,09.08.2019,"22,78","20,69","19,18",18,"17,04","16,24","15,55","14,95","14,43",...,"16,58","16,2","15,83","15,27","14,73","14,19","13,66","13,14","12,67","12,33"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486,11.09.2017,"14,62","12,85","11,76","11,04","10,54","10,18","9,91","9,71","9,57",...,"15,29","14,91","14,56","14,04","13,54","13,08","12,62","12,19","11,78","11,4"
487,08.09.2017,"17,75","15,87","14,55","13,57","12,8","12,18","11,67","11,24","10,89",...,"15,52","15,13","14,76","14,23","13,73","13,24","12,77","12,32","11,87","11,46"
488,07.09.2017,"14,81","13,23","12,11","11,27","10,61","10,08","9,65","9,29",9,...,"15,32","14,94","14,57","14,04","13,53","13,05","12,58","12,14","11,71","11,32"
489,06.09.2017,"10,8","9,58","8,95","8,57","8,31","8,13",8,"7,91","7,84",...,"15,29","14,9","14,53",14,"13,5","13,01","12,55","12,11","11,68","11,3"


Установка даты

In [10]:
valuationDate = ql.Date(12,5,2022)
ql.Settings.instance().evaluationDate = valuationDate
convention = ql.ModifiedFollowing
dayCounter = ql.Actual360()
calendar = ql.TARGET()

Перевод столбца со страйками в число

In [27]:
strikes_ser = strikes_df['SPX10P1W=R']
strikes=strikes_ser.tolist()

In [46]:
for i in range (len(strikes)):
    strikes[i]=strikes[i].replace(',', '.')
    strikes[i]=round(float(strikes[i])/100, 5)

In [48]:
strikes

[27.379,
 27.2594,
 28.2715,
 27.3901,
 28.039,
 28.3431,
 27.5508,
 27.5015,
 26.7317,
 28.3312,
 28.4491,
 28.8876,
 29.4098,
 29.6029,
 29.677,
 29.4074,
 29.6254,
 29.4485,
 29.2063,
 29.0346,
 29.256,
 29.0628,
 29.2674,
 29.3505,
 29.6186,
 29.154,
 29.3224,
 29.1557,
 29.0806,
 29.3907,
 29.4552,
 29.1607,
 28.9682,
 28.6249,
 28.3534,
 28.3158,
 28.3626,
 28.7187,
 28.9019,
 28.9781,
 28.6188,
 28.3785,
 28.1025,
 28.1689,
 28.1023,
 27.964,
 28.0008,
 28.0156,
 27.9231,
 27.654,
 27.4939,
 27.2089,
 26.539,
 26.6081,
 27.0698,
 26.943,
 27.1421,
 27.4697,
 27.3644,
 27.8114,
 27.898,
 27.5193,
 27.8621,
 28.0801,
 27.7811,
 27.3564,
 26.7067,
 27.8034,
 27.3478,
 27.5129,
 27.6274,
 28.3905,
 28.7907,
 28.3436,
 28.3768,
 28.738,
 28.7316,
 28.7974,
 28.6305,
 28.6215,
 28.7634,
 28.4901,
 28.2423,
 28.1929,
 28.5555,
 28.4217,
 28.5504,
 28.3279,
 28.2802,
 28.1152,
 28.3759,
 28.2782,
 28.2141,
 28.1448,
 28.1091,
 28.0918,
 27.6924,
 27.4703,
 27.3164,
 27.4624,
 27.1641,
 

Перевод колонки date в дату

In [64]:
expiration_dates_ser = strikes_df['date']
expiration_dates=expiration_dates_ser.tolist()

In [65]:
for i in range (len(expiration_dates)):
    expiration_dates[i]=datetime.strptime(expiration_dates[i], '%d.%m.%Y').date()

In [66]:
expiration_dates

[datetime.date(2019, 8, 15),
 datetime.date(2019, 8, 14),
 datetime.date(2019, 8, 13),
 datetime.date(2019, 8, 12),
 datetime.date(2019, 8, 9),
 datetime.date(2019, 8, 8),
 datetime.date(2019, 8, 7),
 datetime.date(2019, 8, 6),
 datetime.date(2019, 8, 5),
 datetime.date(2019, 8, 2),
 datetime.date(2019, 8, 1),
 datetime.date(2019, 7, 31),
 datetime.date(2019, 7, 30),
 datetime.date(2019, 7, 29),
 datetime.date(2019, 7, 26),
 datetime.date(2019, 7, 25),
 datetime.date(2019, 7, 24),
 datetime.date(2019, 7, 23),
 datetime.date(2019, 7, 22),
 datetime.date(2019, 7, 19),
 datetime.date(2019, 7, 18),
 datetime.date(2019, 7, 17),
 datetime.date(2019, 7, 16),
 datetime.date(2019, 7, 15),
 datetime.date(2019, 7, 12),
 datetime.date(2019, 7, 11),
 datetime.date(2019, 7, 10),
 datetime.date(2019, 7, 9),
 datetime.date(2019, 7, 8),
 datetime.date(2019, 7, 5),
 datetime.date(2019, 7, 3),
 datetime.date(2019, 7, 2),
 datetime.date(2019, 7, 1),
 datetime.date(2019, 6, 28),
 datetime.date(2019, 6, 27)

Перевод колонки с iv в число

In [43]:
data_ser = iv_df['SPX10P1W=R']
data=data_ser.tolist()

In [44]:
for i in range (len(data)):
    data[i]=data[i].replace(',', '.')
    data[i]=round(float(data[i])/100, 5)

In [51]:
data

[0.2223,
 0.2291,
 0.2161,
 0.2818,
 0.2278,
 0.207,
 0.2543,
 0.2585,
 0.3358,
 0.1942,
 0.2099,
 0.178,
 0.1356,
 0.1145,
 0.1081,
 0.1204,
 0.1083,
 0.1162,
 0.1253,
 0.137,
 0.1291,
 0.1385,
 0.1478,
 0.1526,
 0.0954,
 0.1649,
 0.1194,
 0.1239,
 0.131,
 0.0959,
 0.0982,
 0.1132,
 0.1321,
 0.1554,
 0.1684,
 0.1638,
 0.1617,
 0.146,
 0.1381,
 0.1319,
 0.14,
 0.1755,
 0.1661,
 0.1462,
 0.1671,
 0.1694,
 0.1727,
 0.1739,
 0.1644,
 0.1603,
 0.1578,
 0.1714,
 0.1955,
 0.1924,
 0.1707,
 0.1826,
 0.1833,
 0.1643,
 0.1741,
 0.1517,
 0.1506,
 0.17,
 0.1585,
 0.1478,
 0.1626,
 0.2157,
 0.2726,
 0.21,
 0.2768,
 0.2678,
 0.2514,
 0.1821,
 0.1293,
 0.1625,
 0.1671,
 0.1406,
 0.1342,
 0.1168,
 0.1221,
 0.1281,
 0.1131,
 0.1174,
 0.1266,
 0.1255,
 0.1081,
 0.1287,
 0.1073,
 0.1104,
 0.1258,
 0.1357,
 0.1149,
 0.1297,
 0.116,
 0.121,
 0.1102,
 0.1167,
 0.1326,
 0.1419,
 0.1528,
 0.1487,
 0.1728,
 0.1731,
 0.1295,
 0.1319,
 0.1239,
 0.1175,
 0.1171,
 0.1203,
 0.1124,
 0.1521,
 0.1574,
 0.1898,
 0.19

Модель Хестона

In [68]:
# Определение параметров модели Хестона
theta = 0.01
kappa = 0.01
sigma = 0.01
rho = 0.01
v0 = 0.01

# Границы параметров модели (1=theta, 2=kappa, 3=sigma, 4=rho, 5=v0)
bounds = [(0.01, 1.0), (0.01, 10.0), (0.01, 1.0), (-1.0, 1.0), (0.01, 1.0)]

In [69]:
def HestonPathGenerator(dates, dayCounter, process, nPaths):
    t = np.array([dayCounter.yearFraction(dates[0], d) for d in dates])
    nGridSteps = (t.shape[0] - 1) * 2
    sequenceGenerator = ql.UniformRandomSequenceGenerator(nGridSteps, ql.UniformRandomGenerator())
    gaussianSequenceGenerator = ql.GaussianRandomSequenceGenerator(sequenceGenerator)
    pathGenerator = ql.GaussianMultiPathGenerator(process, t, gaussianSequenceGenerator, False)
    paths = np.zeros(shape = (nPaths, t.shape[0]))
    
    for i in range(nPaths):
        multiPath = pathGenerator.next().value()
        paths[i,:] = np.array(list(multiPath[0]))
        
    return paths